In [ ]:
from datetime import datetime, timezone

import nltk
import pandas as pd
import plotly.express as px
from nrclex import NRCLex
from twarc import Twarc2
from twarc_csv import DataFrameConverter

#nltk.download("punkt")
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
Bearer_Token = "AAAAAAAAAAAAAAAAAAAAAG7PiQEAAAAARqG9r3iYU1cA%2BuEl6pZCpi3DQOw%3DOvnIGTR8sfC297Mb81mGCm0amGLDoPttOA2c0paPea2Mlg0Z5E"

In [3]:
def TweetSearch(query, start_time, end_time):
    client = Twarc2(bearer_token=Bearer_Token)
    converter = DataFrameConverter()

    # The search_recent method call the recent search endpoint to get Tweets based on the query, start and end times
    search_results = client.search_recent(
        query=query, start_time=start_time, end_time=end_time, max_results=50
    )

    # Twarc returns all Tweets for the criteria set above, so we page through the results
    # for count, page in enumerate(search_results):

    Tweets = converter.process(search_results)
    Tweets_str = Tweets.to_string()

    text_object = NRCLex(Tweets_str)
    data = text_object.affect_frequencies

    emotion_df = pd.DataFrame.from_dict(data, orient="index")
    emotion_df = emotion_df.reset_index()
    emotion_df = emotion_df.rename(
        columns={"index": "Emotion Classification", 0: "Emotion Count"}
    )
    emotion_df = emotion_df.sort_values(by=["Emotion Count"], ascending=False)

    emotion_df.drop(
        emotion_df[emotion_df["Emotion Classification"] == "anticip"].index,
        inplace=True,
    )  # This line just fixes a small cosmetic bug in the Twarc library

    fig = px.bar(
        emotion_df,
        x="Emotion Count",
        y="Emotion Classification",
        color="Emotion Classification",
        orientation="h",
        width=800,
        height=400,
    )
    fig.show()

    return

In [4]:
# Specify your query, like the stock ticker or name of the company you are investigating
query = "AAPL lang:en"

# Specify the start and end times in UTC for the time period you want Tweets from
# These dates and times must be within the past week!
start_time = datetime(2022, 4, 20, 0, 0, 0, 0, timezone.utc)
end_time = datetime(2022, 4, 20, 1, 0, 0, 0, timezone.utc)

scrape_new_data = False

if scrape_new_data:
    df_aapl = TweetSearch(query, start_time, end_time)
else:
    df_aapl = pd.read_csv("AAPL_lang_en_emotion.csv")

df_aapl

FileNotFoundError: [Errno 2] No such file or directory: 'AAPL_lang_en_emotion.csv'

In [ ]:
# Specify your query, like the stock ticker or name of the company you are investigating
query = "IBM lang:en"

# Be default we are using the same start and end time.
# This is also intentional because we want a good comparison

if scrape_new_data:
    df_ibm = TweetSearch(query, start_time, end_time)
else:
    df_ibm = pd.read_csv("IBM_lang_en_emotion.csv")

df_ibm